In [1]:
import pandas as pd
import numpy as np
import math
import random


ignore_features = ['58', '16', '7', '25', '58', '23', '20']

toencode = {}
f = open("field_types.txt", 'r')
for line in f.readlines():
    line = line.strip().split(" ")
    if line[1] != "numeric" and line[0] not in ignore_features:
        toencode[line[0]] = len(line) - 1
        

f1 = open("data.csv")
data_df = pd.read_csv(f1, sep=',', header = 'infer', low_memory=False)
data_df = data_df.drop(ignore_features, axis=1)
label_df = data_df[['label']]
data_df = data_df.drop('label', axis=1)
label = np.array(label_df)


f2 = open("quiz.csv")
quiz_df = pd.read_csv(f2, sep=',', header = 'infer', low_memory=False)
quiz_df = quiz_df.drop(ignore_features, axis=1)



In [2]:
dat_trans = pd.get_dummies(data_df, columns=toencode.keys(), dummy_na=False)
quiz_trans = pd.get_dummies(data_df, columns=toencode.keys(), dummy_na=False)

In [3]:
data = np.array(dat_trans)
quiz = np.array(quiz_trans)

In [4]:
def splitdata(data, label):
    n = data.shape[0]
    train_num = int(math.ceil(n*.8))
    indices = random.sample(xrange(n), n)

    training_idx, test_idx = indices[train_num:], indices[:train_num]
    training, test = data[training_idx,:], data[test_idx,:]
    lab, testlabel = label[training_idx,:], label[test_idx,:]
    
    return training, lab, test, testlabel

train, lab, test, testlabel = splitdata(data, label)

In [6]:
def knn(train, trainlabels, test, k):
    if k/2 == 0.0:
        print "K must be an odd number to break ties."
        return None
    preds = []
    for t in test:
        distance = []
        for s in train:
            x = s - t
            dis = np.dot(x, x)
            distance.append(dis)
        i = 0
        n_list = []
        while i < k:
            neighbor = np.argmin(distance)
            distance[neighbor] = 1000000
            n_list.append(neighbor)
            i += 1
        p = trainlabels[n_list]
        pp = np.sum(p) / abs(np.sum(p))
        
        preds.append(pp)
    return preds

In [ ]:
preds = knn(train, lab, test, 3)
print eval(preds, testlabel)

In [7]:
def eval(preds, testlabels):
    if len(preds) == len(testlabels):
        error = np.count_nonzero(preds != testlabels) / np.float(len(preds))
        return 1 - error
    


In [5]:
def knn2(train, trainlabels, test):
    norms = np.square(np.linalg.norm(train, axis = 1))
    cross = np.dot(test, train.T)
    distances = -2*cross + norms
    indicies = np.argmin(distances, axis = 1)
    preds = trainlabels[indicies]
    return preds

In [ ]:
preds = knn2(train, lab, test)
print eval(preds, testlabel)

In [14]:
preds = knn2(train, lab, quiz[0:10000])
print eval(preds, testlabel[0:10000])

0.493


In [44]:
from sklearn.neighbors import KNeighborsClassifier
lab = np.ravel(lab)
neigh = KNeighborsClassifier(n_neighbors=2, algorithm='ball_tree')
f = neigh.fit(train, lab) 

In [45]:
preds = neigh.predict(test)
print eval(preds, testlab)

0.857691928649
